# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['employees', 'salary', 'studies'],
        'definition': ['Employee information, name...', 'Salary details for each year', 'Educational studies, name of the institution']} 
df = pd.DataFrame(data)
print(df)

       table                                    definition
0  employees                 Employee information, name...
1     salary                  Salary details for each year
2    studies  Educational studies, name of the institution


In [4]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [5]:
print(text_tables)

employees: Employee information, name...
salary: Salary details for each year
studies: Educational studies, name of the institution


In [6]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [7]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='What is the highest salary in compare to studies')

In [8]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "salary", "studies"]
}
```


In [9]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question = 'What is the highest salary')

In [10]:
print(return_OAI(pqt3))

```json
{
    "tables": ["salary"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [12]:
# Import necessary libraries
import pandas as pd

# Define tables with their names and short descriptions
data = {
    'table': ['employees', 'salary', 'studies', 'departments'],
    'definition': [
        'Employee information, including names and positions.',
        'Salary details, including yearly earnings and bonuses.',
        'Educational background, including institutions and degrees.',
        'Department details, including names and department heads.'
    ]
}
df = pd.DataFrame(data)

# Create text descriptions of the tables
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

# Define the prompt template
prompt_question_tables = """
Given the following tables and their content definitions,
### Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.
### User Question:
{question}
"""

# Function to query GPT (replace with your API interaction function)
def query_gpt(prompt):
    # This should call the GPT API with the prompt and return the response.
    # Replace this stub with your GPT function, e.g., return_OAI(prompt)
    print(f"Querying GPT with prompt:\n{prompt}\n")
    return {"response": "Sample response"}  # Example response

# Example questions to test
questions = [
    "What is the highest salary across all employees?",
    "Which department has the most employees?",
    "What is the average salary for employees with a Master's degree?",
    "Which department head has the highest-paid team?"
]

# Loop through the questions and query GPT
for question in questions:
    prompt = prompt_question_tables.format(tables=text_tables, question=question)
    response = query_gpt(prompt)
    print(f"Question: {question}")
    print(f"Response: {response}")
    print("\n")

# Instructions for further exploration
print("Experiment with different questions and evaluate the GPT responses.")

Querying GPT with prompt:

Given the following tables and their content definitions,
### Tables
employees: Employee information, including names and positions.
salary: Salary details, including yearly earnings and bonuses.
studies: Educational background, including institutions and degrees.
departments: Department details, including names and department heads.

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a JSON format.
### User Question:
What is the highest salary across all employees?


Question: What is the highest salary across all employees?
Response: {'response': 'Sample response'}


Querying GPT with prompt:

Given the following tables and their content definitions,
### Tables
employees: Employee information, including names and positions.
salary: Salary details, including yearly earnings and bonuses.
studies: Educational background, including institutions and degrees.
departments: Department details, i